In [ ]:
!pip install -r requirements.txt

In [5]:
import numpy as np
import time
import imutils
import cv2 




confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

inpWidth = 416  #608     #Width of network's input image
inpHeight = 416 #608     #Height of network's input image


LABELS ="C:/Users/karim/Desktop/ANPR-master/Licence_plate_detection/classes.names";

# Give the configuration and weight files for the model and load the network using them.

modelConfiguration = "C:/Users/karim/Desktop/License-Plate-Detection-Recognition/Licence_plate_detection/darknet-yolov3.cfg"
modelWeights = "C:/Users/karim/Desktop/License-Plate-Detection-Recognition/Licence_plate_detection//lapi.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
dtype="uint8")

# initialize the video stream, pointer to output video file, and
# frame dimensions
#Load the video Located inside video Folder
vs = cv2.VideoCapture('C:/Users/karim/downloads/v/vid1.mp4')
writer = None
(W, H) = (None, None)
# try to determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))    
    print("[INFO] {} total frames in video".format(total))
# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > confThreshold:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                print(confidences)
                print(boxes)
                
    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold,
        nmsThreshold)
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('../video/car_chase_02.avi', fourcc, 30,
            (frame.shape[1], frame.shape[0]), True)
                # some information on processing single frame
        if total > 0:
            elap = (end - start)
            print("[INFO] single frame took {:.4f} seconds".format(elap))
            print("[INFO] estimated total time to finish: {:.4f}".format(
                elap * total))
    # write the output frame to disk
    writer.write(frame)
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] 537 total frames in video
[INFO] single frame took 2.5259 seconds
[INFO] estimated total time to finish: 1356.4212
[0.8955681324005127]
[[557, 500, 100, 28]]
[0.9016232490539551]
[[557, 500, 100, 28]]
[0.9019862413406372]
[[557, 500, 100, 28]]
[0.9983994960784912]
[[527, 511, 94, 29]]
[0.9983822703361511]
[[527, 511, 94, 29]]
[0.9983983039855957]
[[527, 511, 94, 29]]
[0.9994738698005676]
[[494, 527, 98, 30]]
[0.9994804263114929]
[[494, 527, 98, 30]]
[0.9994786381721497]
[[494, 527, 98, 30]]
[0.9984846711158752]
[[450, 543, 108, 33]]
[0.9985032677650452]
[[450, 543, 108, 33]]
[0.9985091090202332]
[[450, 543, 108, 33]]
[0.9998157024383545]
[[398, 562, 115, 35]]
[0.9998157024383545]
[[398, 562, 115, 36]]


KeyboardInterrupt: 